In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
from random import shuffle
from tqdm import tqdm
import os

# Display entire width of browser
from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [3]:
def extractImages(pathIn, pathOut):
    count = 0
    vidcap = cv.VideoCapture(pathIn)
    success,image = vidcap.read()
    success = True
    while success:
      vidcap.set(cv.CAP_PROP_POS_MSEC,(count*1000))    # added this line 
      success,image = vidcap.read()
      print('Read a new frame: ', success)     
      cv.imwrite( pathOut + "/frame%d.jpg" % count, image)     # save frame as JPEG file
      count = count + 1

In [4]:
w = 96
h = 96

In [5]:
def list_dir(path):
    fileNames = []
    for root, dirs, files in os.walk(path):  
        for filename in files: 
            fileNames.append(filename)            
    return fileNames

In [6]:
def invert(img):
    inverted = img[:,:,::-1]
    inverted[np.where((inverted > [127,127,127]).all(axis = 2))] = [0,0,0]
    return inverted

In [7]:
files = list_dir('../videos')

In [8]:
print(files[0])

video-1.mp4


In [9]:
def extract(videoRoot, imageRoot):
    videos = list_dir(videoRoot)
    for video in videos:
        videoPath = videoRoot + '/' + video
        videoName = video.split('.')[0]
        imagePath = imageRoot+'/' +videoName
        if not os.path.exists(imagePath):
            os.makedirs(imagePath)
            extractImages(videoPath,imagePath)        

In [10]:
extract('../videos', '../images')

In [11]:
labelsFile = '../labels/labels.csv'
def read_labels(lablesFile):
    labelData = []
    with open(lablesFile) as inf:
        for line in tqdm(inf):            
            video, frame, label, clazz = line.strip().split(",")
            frame = int(frame)
            label = int(label)
            labelData.append({'video':video, 'frame':frame, 'label':label, 'class':clazz})
    return labelData

labels = read_labels(labelsFile)
            

264it [00:00, 131883.78it/s]


In [12]:
def pre_process(img):
    hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    lower = np.array([49,89,127])
    upper = np.array([179,225,255])
    mask = cv.inRange(hsv, lower, upper)
    result = cv.bitwise_and(hsv, hsv, mask=mask)
    #result_gray = cv.cvtColor(result, cv.COLOR_BGR2GRAY)
    _, thresh = cv.threshold(result, 7, 255, cv.THRESH_BINARY)
    resized = cv.resize(thresh, (96,96))    
    #cropped = resized[0:72, 0:96]
    return resized

In [13]:
def get_label(file, labels):    
    for l in labels:         
        if 'frame' + str(l['frame']) + '.jpg' == file:
            if l['class'] == 'Light':                
                return np.array([1,0])
            else:
                return np.array([0,1])
    return np.array([1,1])
    
def train_data_with_label(train_data, labels):
    train_images = []    
    for i in tqdm(os.listdir(train_data)):        
        path = os.path.join(train_data, i)        
        img = cv.imread(path)
        processed = pre_process(img)     
        label = get_label(i, labels)      
        train_images.append([np.array(processed),label])              
    #shuffle(train_images)
    return train_images

In [14]:
train_data = train_data_with_label('../images/video-1', labels)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 265/265 [00:08<00:00, 29.98it/s]


In [15]:
train_data[100]

[array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=uint8), array([0, 1])]

In [16]:
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *

Using TensorFlow backend.


In [17]:
test_data = train_data_with_label('../images/video-1', labels)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 265/265 [00:08<00:00, 30.15it/s]


In [18]:
tr_img_data = np.array([i[0] for i in train_data]).reshape(-1, w,h,1)
tr_lbl_data = np.array([i[1] for i in train_data])

tst_img_data = np.array([i[0] for i in test_data]).reshape(-1, w,h,1)
tst_lbl_data = np.array([i[1] for i in test_data])


In [20]:
model = Sequential()

model.add(InputLayer(input_shape=[w,h,1]))
model.add(Conv2D(filters=32, kernel_size=5,strides=1,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=5,padding='same'))

model.add(Conv2D(filters=50,kernel_size=5,strides=1,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=5,padding='same'))

model.add(Conv2D(filters=80,kernel_size=5,strides=1,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=5,padding='same'))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(2,activation='softmax'))
optimizer = Adam(lr=1e-3)

model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x=tr_img_data,y=tr_lbl_data,epochs=50,batch_size=10)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


ValueError: Input arrays should have the same number of samples as target arrays. Found 795 input samples and 265 target samples.

In [ ]:
    test_img = train_data[135][0]
    predict_data = test_img.reshape(-1,w,h,1)
    model_out = model.predict([predict_data])
    print(model_out)

In [ ]:
fig =plt.figure(figsize=(14,14))
plot_cols = 5
plot_rows = len(train_data)/plot_cols
for cnt, data in enumerate(train_data[:]):
    y = fig.add_subplot(plot_rows,plot_cols, cnt+1)
    img = data[0]
    data = img.reshape(1,w,h,1)
    model_out = model.predict([data])
    
    if np.argmax(model_out) == 1:
        str_label = 'No Lights'
    else:
        str_label = 'Lights'
        
    y.imshow(img, cmap='gray')
    plt.title(str_label)
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)    